In [1]:
from data_utils import *

In [ ]:
def import_op_dataset(operator, operand_digits, train_ratio, dev_ratio, test_ratio):
    # Path of op_dataset
    import_path = '{}/{}-bit/{}/op_dataset.pickle'.format(config.dir_data(), operand_digits, operator)

    # Import the op_dataset
    with open(import_path, 'rb') as f:
        op_dataset = pickle.load(f)

    # Dataset size
    ds_size = op_dataset['input'].shape[0]

    # Shuffle input and output data
    randomize = np.arange(ds_size)
    np.random.shuffle(randomize)
    op_dataset['input'] = op_dataset['input'][randomize]
    op_dataset['output'] = op_dataset['output'][randomize]

    # Make a training set.
    train_end_index = int(ds_size * train_ratio)
    input_train = op_dataset['input'][:train_end_index,:]
    target_train = op_dataset['output'][:train_end_index,:]

    # Make a development set.
    dev_end_index = int(ds_size * (train_ratio + dev_ratio))

    if dev_ratio != 0:
        input_dev = op_dataset['input'][:dev_end_index,:]
        target_dev = op_dataset['output'][:dev_end_index,:]
    else:
        input_dev = None
        target_dev = None

    # Maek a test set.
    input_test = op_dataset['input'][dev_end_index:,:]
    target_test = op_dataset['output'][dev_end_index:,:]

    return (input_train, input_dev, input_test,
            target_train, target_dev, target_test)

# Start to implement

In [2]:
operator = 'add'
operand_digits = 4 
(train_ratio, dev_ratio, test_ratio) = (0.8*0.8, 0.8*0.2, 0.2)

In [3]:
print((train_ratio, dev_ratio, test_ratio) )

(0.6400000000000001, 0.16000000000000003, 0.2)


In [4]:
# Path of op_dataset
import_path = '{}/{}-bit/{}/op_dataset.pickle'.format(config.dir_data(), operand_digits, operator)

In [5]:
# Import the op_dataset
with open(import_path, 'rb') as f:
    op_dataset = pickle.load(f)

In [6]:
# Dataset size
ds_size = op_dataset['input'].shape[0]

In [7]:
# Shuffle input and output data
randomize = np.arange(ds_size)
np.random.shuffle(randomize)
op_dataset['input'] = op_dataset['input'][randomize]
op_dataset['output'] = op_dataset['output'][randomize]

In [14]:
import_path = 'data/{}-bit/{}/carry_datasets.pickle'.format(operand_digits, operator)

with open(import_path, 'rb') as f:
    carry_datasets = pickle.load(f)

In [23]:
carry_datasets[carries]['input']

array([[0, 0, 0, 1, 1, 1, 1, 1],
       [0, 0, 1, 1, 1, 1, 0, 1],
       [0, 0, 1, 1, 1, 1, 1, 1],
       [0, 1, 0, 1, 1, 0, 1, 1],
       [0, 1, 0, 1, 1, 1, 1, 1],
       [0, 1, 1, 1, 1, 0, 0, 1],
       [0, 1, 1, 1, 1, 0, 1, 1],
       [0, 1, 1, 1, 1, 1, 0, 1],
       [0, 1, 1, 1, 1, 1, 1, 1],
       [1, 0, 0, 1, 0, 1, 1, 1],
       [1, 0, 0, 1, 1, 1, 1, 1],
       [1, 0, 1, 1, 0, 1, 0, 1],
       [1, 0, 1, 1, 0, 1, 1, 1],
       [1, 0, 1, 1, 1, 1, 0, 1],
       [1, 0, 1, 1, 1, 1, 1, 1],
       [1, 1, 0, 1, 0, 0, 1, 1],
       [1, 1, 0, 1, 0, 1, 1, 1],
       [1, 1, 0, 1, 1, 0, 1, 1],
       [1, 1, 0, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 0, 0, 0, 1],
       [1, 1, 1, 1, 0, 0, 1, 1],
       [1, 1, 1, 1, 0, 1, 0, 1],
       [1, 1, 1, 1, 0, 1, 1, 1],
       [1, 1, 1, 1, 1, 0, 0, 1],
       [1, 1, 1, 1, 1, 0, 1, 1],
       [1, 1, 1, 1, 1, 1, 0, 1],
       [1, 1, 1, 1, 1, 1, 1, 1]])

In [38]:
len(carry_datasets[0]['input'])

81

In [40]:
np.concatenate((carry_datasets[0]['input'], carry_datasets[0]['input']), axis=0).shape

(162, 8)

In [60]:
input_train_list = list()
input_dev_list = list()
input_test_list = list()
target_train_list = list()
target_dev_list = list()
target_test_list = list()

splited_carry_datasets = dict()

for carries in carry_datasets.keys():
    # Initialize a dict for the number of carries
    splited_carry_datasets[carries] = dict()
    splited_carry_datasets[carries]['input'] = dict()
    splited_carry_datasets[carries]['output'] = dict()
    
    # Get the size of a carry dataset
    carry_ds_size = carry_datasets[carries]['input'].shape[0]
    print(carry_ds_size)

    # Shuffle input and output data
    randomize = np.arange(carry_ds_size)
    np.random.shuffle(randomize)
    carry_datasets[carries]['input'] = carry_datasets[carries]['input'][randomize]
    carry_datasets[carries]['output'] = carry_datasets[carries]['output'][randomize]

    # Make a training set.
    train_end_index = int(carry_ds_size * train_ratio)
    input_train = carry_datasets[carries]['input'][:train_end_index,:]
    target_train = carry_datasets[carries]['output'][:train_end_index,:]
    input_train_list.append(input_train)
    target_train_list.append(target_train)
    splited_carry_datasets[carries]['input']['train'] = input_train
    splited_carry_datasets[carries]['output']['train'] = target_train
    
    
    # Make a development set.
    dev_end_index = int(carry_ds_size * (train_ratio + dev_ratio))

    if dev_ratio != 0:
        input_dev = carry_datasets[carries]['input'][train_end_index:dev_end_index,:]
        target_dev = carry_datasets[carries]['output'][train_end_index:dev_end_index,:]
        input_dev_list.append(input_dev)
        target_dev_list.append(target_dev)
        splited_carry_datasets[carries]['input']['dev'] = input_dev
        splited_carry_datasets[carries]['output']['dev'] = target_dev
    else:
        input_dev = None
        target_dev = None
    
    # Maek a test set.
    input_test = carry_datasets[carries]['input'][dev_end_index:,:]
    target_test = carry_datasets[carries]['output'][dev_end_index:,:]
    input_test_list.append(input_test)
    target_test_list.append(target_test)
    splited_carry_datasets[carries]['input']['test'] = input_test
    splited_carry_datasets[carries]['output']['test'] = target_test
    
input_train = np.concatenate(input_train_list, axis=0)
input_dev = np.concatenate(input_dev_list, axis=0)
input_test = np.concatenate(input_test_list, axis=0)
target_train = np.concatenate(target_train_list, axis=0)
target_dev = np.concatenate(target_dev_list, axis=0)
target_test = np.concatenate(target_test_list, axis=0)

81
54
52
42
27


In [68]:
summ = 0
for carries in splited_carry_datasets.keys():
    tmp_sum = 0
    tmp_sum += splited_carry_datasets[carries]['input']['train'].shape[0]
    tmp_sum += splited_carry_datasets[carries]['input']['dev'].shape[0]
    tmp_sum += splited_carry_datasets[carries]['input']['test'].shape[0]
    print(tmp_sum)

    summ += splited_carry_datasets[carries]['input']['train'].shape[0]
    summ += splited_carry_datasets[carries]['input']['dev'].shape[0]
    summ += splited_carry_datasets[carries]['input']['test'].shape[0]

print(summ)

81
54
52
42
27
256


In [71]:
def import_op_dataset(operator, operand_digits, train_ratio, dev_ratio, test_ratio):
    input_train_list = list()
    input_dev_list = list()
    input_test_list = list()
    target_train_list = list()
    target_dev_list = list()
    target_test_list = list()

    splited_carry_datasets = dict()

    for carries in carry_datasets.keys():
        # Initialize a dict for the number of carries
        splited_carry_datasets[carries] = dict()
        splited_carry_datasets[carries]['input'] = dict()
        splited_carry_datasets[carries]['output'] = dict()

        # Get the size of a carry dataset
        carry_ds_size = carry_datasets[carries]['input'].shape[0]

        # Shuffle input and output data
        randomize = np.arange(carry_ds_size)
        np.random.shuffle(randomize)
        carry_datasets[carries]['input'] = carry_datasets[carries]['input'][randomize]
        carry_datasets[carries]['output'] = carry_datasets[carries]['output'][randomize]

        # Make a training set.
        train_end_index = int(carry_ds_size * train_ratio)
        input_train = carry_datasets[carries]['input'][:train_end_index,:]
        target_train = carry_datasets[carries]['output'][:train_end_index,:]
        input_train_list.append(input_train)
        target_train_list.append(target_train)
        splited_carry_datasets[carries]['input']['train'] = input_train
        splited_carry_datasets[carries]['output']['train'] = target_train


        # Make a development set.
        dev_end_index = int(carry_ds_size * (train_ratio + dev_ratio))

        if dev_ratio != 0:
            input_dev = carry_datasets[carries]['input'][train_end_index:dev_end_index,:]
            target_dev = carry_datasets[carries]['output'][train_end_index:dev_end_index,:]
            input_dev_list.append(input_dev)
            target_dev_list.append(target_dev)
            splited_carry_datasets[carries]['input']['dev'] = input_dev
            splited_carry_datasets[carries]['output']['dev'] = target_dev
        else:
            input_dev = None
            target_dev = None

        # Maek a test set.
        input_test = carry_datasets[carries]['input'][dev_end_index:,:]
        target_test = carry_datasets[carries]['output'][dev_end_index:,:]
        input_test_list.append(input_test)
        target_test_list.append(target_test)
        splited_carry_datasets[carries]['input']['test'] = input_test
        splited_carry_datasets[carries]['output']['test'] = target_test

    input_train = np.concatenate(input_train_list, axis=0)
    input_dev = np.concatenate(input_dev_list, axis=0)
    input_test = np.concatenate(input_test_list, axis=0)
    target_train = np.concatenate(target_train_list, axis=0)
    target_dev = np.concatenate(target_dev_list, axis=0)
    target_test = np.concatenate(target_test_list, axis=0)
    
    return (input_train, input_dev, input_test,
            target_train, target_dev, target_test,
            splited_carry_datasets)

In [74]:
outputs = (input_train, input_dev, input_test,
            target_train, target_dev, target_test,
            splited_carry_datasets) = import_op_dataset('add', 4, 0.6, 0.2, 0.2)

In [76]:
for output in (input_train, input_dev, input_test, target_train, target_dev, target_test):
    print(output.shape)

(152, 8)
(50, 8)
(54, 8)
(152, 8)
(50, 8)
(54, 8)


In [77]:
splited_carry_datasets.keys()

dict_keys([0, 1, 2, 3, 4])

In [87]:
summ = 0
for carries in splited_carry_datasets.keys():
    summ += splited_carry_datasets[carries]['input']['train'].shape[0]
print(summ)

152


In [88]:
summ = 0
for carries in splited_carry_datasets.keys():
    summ += splited_carry_datasets[carries]['input']['dev'].shape[0]
print(summ)

50


In [89]:
summ = 0
for carries in splited_carry_datasets.keys():
    summ += splited_carry_datasets[carries]['input']['test'].shape[0]
print(summ)

54
